**Pregunta N°2 - Segundo Parcial - SIS420**

**Introducción.**

En este cuadernillo se realizará el aprendizaje por refuerzo, esto aplicado a un cuatro en raya.

**Objetivo.**

Generar las tablas respectivas para que el modelo aprenda y logre jugar al cuatro en raya.



# Creación de los elementos del Juego y del Agente

**Modelo del entorno:** El entorno es todo aquello con lo que el agente interactúa y sobre lo cual tiene poco o ningún control directo. En el código, la clase Board representa el entorno porque mantiene el estado del juego, que es externo a los agentes y dicta las reglas del juego. Los agentes no pueden cambiar las reglas, solo pueden actuar dentro de ellas. Por lo tanto, Board es el modelo del entorno ya que define cómo se representa el estado del juego (el tablero de 7x6), las acciones válidas (movimientos posibles) y las condiciones de terminación del juego (victoria, derrota o empate).

**Agente:** Un agente es una entidad que toma decisiones y realiza acciones dentro del entorno para alcanzar un objetivo. En el código, la clase Agent es el agente porque es responsable de decidir qué movimientos realizar en el tablero. El agente tiene una política (definida en su método move) que utiliza para seleccionar acciones, y una función de valor que utiliza para estimar qué tan buenas son las posiciones en el tablero. El agente aprende de la experiencia al actualizar su función de valor basada en las recompensas recibidas, lo que le permite mejorar sus decisiones a lo largo del tiempo.

**Recompensa:** La recompensa es una señal que el agente recibe del entorno para evaluar la calidad de sus acciones. En el código, la recompensa se da en el método reward de la clase Game. Esta señal indica al agente si la acción que tomó condujo a un resultado positivo (ganar el juego), negativo (perder el juego) o neutral (empatar). La recompensa es fundamental en el aprendizaje por refuerzo porque guía al agente para que refuerce las acciones que conducen a resultados positivos y evite las que conducen a resultados negativos.

**Política:** La política es la estrategia que el agente utiliza para decidir qué acción tomar en un estado dado. En el código, la política se implementa en el método move de la clase Agent. La política puede ser determinista (siempre elige la misma acción para un estado dado) o estocástica (elige acciones basadas en probabilidades). La política en este código es estocástica ya que el agente puede explorar (elegir una acción al azar) o explotar (elegir la mejor acción según su función de valor), y la probabilidad de explorar se controla con prob_exp.

**Función de valor:** La función de valor es una estimación del agente de qué tan buena es una posición en el tablero, dadas las recompensas futuras que podría recibir. En el código, la función de valor está representada por el diccionario value_function en la clase Agent. Esta función asigna un valor numérico a cada estado posible del tablero, que se actualiza a medida que el agente recibe recompensas. La función de valor ayuda al agente a predecir qué estados son más prometedores y, por lo tanto, a elegir acciones que maximicen las recompensas futuras.

In [9]:
"""
Creación de las librerías para trabajar con este Agente para un 4 en raya
"""

import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [10]:
class Board():
    def __init__(self):
        # El tablero debe ser de 6x7
        self.state = np.zeros((6, 7))

    def valid_moves(self):
        return [j for j in range(7) if self.state[0, j] == 0]

    def update(self, symbol, col):
        # Encuentra la fila más baja disponible en la columna seleccionada
        for row in range(5, -1, -1):  # Comenzar desde la parte inferior del tablero
            if self.state[row, col] == 0:
                self.state[row, col] = symbol
                return
        raise ValueError("Movimiento ilegal!")

    def is_game_over(self):
        # Verificar filas y columnas para encontrar cuatro en línea
        for c in range(7):
            for r in range(6):
                if r <= 2 and np.all(self.state[r:r+4, c] == self.state[r, c]) and self.state[r, c] != 0:
                    return self.state[r, c]
                if c <= 3 and np.all(self.state[r, c:c+4] == self.state[r, c]) and self.state[r, c] != 0:
                    return self.state[r, c]
        # Verificar diagonales
        for r in range(3):
            for c in range(4):
                if np.all([self.state[r+i, c+i] == self.state[r, c] for i in range(4)]) and self.state[r, c] != 0:
                    return self.state[r, c]
                if np.all([self.state[r+3-i, c+i] == self.state[r+3, c] for i in range(4)]) and self.state[r+3, c] != 0:
                    return self.state[r+3, c]
        # Si no se encuentra un ganador, retornar None
        return None

    def reset(self):
        self.state = np.zeros((6, 7))

class Agent:
    def __init__(self, alpha=0.1, prob_exp=0.3, symbol=1):
        self.alpha = alpha
        self.prob_exp = prob_exp
        self.value_function = {}
        self.positions = []
        self.symbol = symbol

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploración
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            col = np.random.choice(valid_moves)
            return col
        # explotación
        # vamos a la posición con más valor
        max_value = -np.inf
        best_col = None
        for col in valid_moves:
            for row in range(5, -1, -1):
                if board.state[row, col] == 0:
                    next_board = board.state.copy()
                    next_board[row, col] = self.symbol
                    next_state = str(next_board.reshape(6*7))
                    value = self.value_function.get(next_state, 0)
                    if value > max_value:
                        max_value = value
                        best_col = col
                    break
        return best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(6*7)))

    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

class Game:
    def __init__(self, agent1, agent2):
        self.board = Board()
        self.agent1 = agent1
        self.agent2 = agent2

    def selfplay(self, rounds):
        wins = {1: 0, 2: 0, 'draw': 0}
        for _ in range(rounds):
            self.board.reset()
            game_over = False
            turn = 0
            while not game_over:
                player = self.agent1 if turn % 2 == 0 else self.agent2
                col = player.move(self.board)
                self.board.update(player.symbol, col)
                player.update(self.board)
                winner = self.board.is_game_over()
                if winner is not None:
                    wins[winner] += 1
                    game_over = True
                elif all(self.board.state[0, :] != 0):
                    wins['draw'] += 1
                    game_over = True
                turn += 1
        return wins

In [11]:
# Creación de agentes y juego
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()
game = Game(agent1, agent2)
wins = game.selfplay(30000)

# Guardar y mostrar la función de valor aprendida
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})
print(tabla)

# Guardar la función de valor en un archivo
with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)

Empty DataFrame
Columns: [estado, valor]
Index: []


In [13]:
tabla

,estado,valor
